In [1]:
# %pip install -q diffusers transformers xformers accelerate gradio ray numba safetensors
# %pip install -q numpy scipy ftfy imageio matplotlib Pillow
# %pip install -q python-dotenv

In [2]:
from dotenv import load_dotenv
load_dotenv()

import os
os.environ['HUGGINGFACE_HUB_CACHE'] = 'D:\\Code\\Huggingface_cache\\'
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
import torch
import ray
import gc
import numpy as np
import einops

import datetime
import pytz
import imageio
import matplotlib.pyplot as plt

from PIL import Image
from tqdm.auto import tqdm

from numba import cuda as numba_cuda
from diffusers import StableDiffusionPipeline, StableDiffusionInpaintPipeline, StableDiffusionImg2ImgPipeline, CycleDiffusionPipeline, StableDiffusionDepth2ImgPipeline
from diffusers import DDIMScheduler, PNDMScheduler, LMSDiscreteScheduler, DPMSolverMultistepScheduler, EulerAncestralDiscreteScheduler, EulerDiscreteScheduler
from diffusers import AutoencoderKL, UNet2DConditionModel
from diffusers.pipelines.stable_diffusion.safety_checker import StableDiffusionSafetyChecker

from accelerate.hooks import remove_hook_from_submodules

from xformers.ops import MemoryEfficientAttentionFlashAttentionOp
from transformers import CLIPTextModel, CLIPTokenizer
from transformers import AutoFeatureExtractor
from transformers import logging
logging.set_verbosity_error()

ray.init()

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
2023-04-06 15:02:35,670	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.8.7
Ray version:,2.3.1


In [4]:
# get token: https://huggingface.co/settings/tokens
HF_TOKEN = "YOUR_HUGGING_FACE_TOKEN"

In [5]:
print(torch.cuda.is_available())

True


In [6]:
#optimization https://huggingface.co/docs/diffusers/optimization/fp16#memory-efficient-attention
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

In [7]:
@ray.remote(num_gpus=1)   
class StableDiffusionInterface(): 
    def __init__(self, model_name, scheduler, use_auth_token, torch_dtype = torch.float32, safe_mode = False, device = "cuda", revision = "fp16",):
        self.pipe = StableDiffusionPipeline.from_pretrained(
            model_name, 
            scheduler = scheduler,  
            torch_dtype = torch_dtype,  
            use_auth_token = use_auth_token,
        )
        self.pipe.safety_checker = None

        #optimization
        self.pipe.enable_model_cpu_offload()
        self.pipe.enable_attention_slicing(1)
        self.pipe.unet.to(memory_format=torch.channels_last)
        self.pipe.enable_vae_slicing()
        self.pipe.enable_vae_tiling()
        self.pipe.enable_xformers_memory_efficient_attention()

        # try:
        #     if not safe_mode: #Not working with stable diffusion 2.0
        #         self.pipe.safety_checker = lambda images, clip_input: (images, False)
        # except:
        #     pass

        # self.pipe.to(device)

    def remove_hooks(self):
        self.pipe.disable_xformers_memory_efficient_attention()
        self.pipe.disable_attention_slicing()
        remove_hook_from_submodules(self.pipe.vae)
        remove_hook_from_submodules(self.pipe.text_encoder)
        remove_hook_from_submodules(self.pipe.unet)

    def name(self):
        return "StableDiffusionInterface"
   
    def __call__(self, prompt = "", height = 64, width = 64, negative_prompt = "", num_images_per_prompt = 1, num_inference_steps = 50, guidance_scale = 7.5, seed = None):
        g_cuda = None
        if seed is not None:
            g_cuda = torch.Generator(device='cuda')
            g_cuda.manual_seed(seed)

        return self.pipe(
            prompt,
            height=height,
            width=width,
            negative_prompt=negative_prompt,
            num_images_per_prompt=num_images_per_prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator = g_cuda
        ).images

In [8]:
@ray.remote(num_gpus=1)   
class CycleDiffusionInterface:
    def __init__(self, model_name, scheduler, use_auth_token, torch_dtype = torch.float32, safe_mode = False, device = "cuda", revision = "fp16",):
        self.pipe = CycleDiffusionPipeline.from_pretrained(
            model_name, 
            scheduler = scheduler,  
            torch_dtype=torch_dtype,  
            use_auth_token = use_auth_token,
            # revision=revision, 
        )
        self.pipe.safety_checker = None

        self.pipe.enable_model_cpu_offload()
        self.pipe.enable_attention_slicing(1)
        self.pipe.unet.to(memory_format=torch.channels_last)
        # self.pipe.enable_vae_slicing() #Not working with cycle stable diffusion pipeline
        # self.pipe.enable_vae_tiling() #Not working with cycle stable diffusion pipeline
        self.pipe.enable_xformers_memory_efficient_attention()

    def remove_hooks(self):
        self.pipe.disable_xformers_memory_efficient_attention()
        self.pipe.disable_attention_slicing()
        remove_hook_from_submodules(self.pipe.vae)
        remove_hook_from_submodules(self.pipe.text_encoder)
        remove_hook_from_submodules(self.pipe.unet)

    def name(self):
        return "CycleDiffusionInterface"
    
    def __call__(
            self, 
            prompt = "",  
            source_prompt = "", 
            image = None, 
            height = 64, 
            width = 64,  
            num_images_per_prompt = 1, 
            num_inference_steps = 50, 
            eta=0.1,
            strength=0.85,
            guidance_scale = 7.5, 
            source_guidance_scale=1,
            seed = None):
        
        g_cuda = None
        if seed is not None:
            g_cuda = torch.Generator(device='cuda')
            g_cuda.manual_seed(seed)

        return self.pipe(
            prompt,
            source_prompt = source_prompt,
            image = image.resize((width, height)),
            num_images_per_prompt=num_images_per_prompt,
            num_inference_steps=num_inference_steps,
            eta=eta,
            strength=strength,
            guidance_scale=guidance_scale,
            source_guidance_scale=source_guidance_scale,
            generator = g_cuda
        ).images

In [9]:
@ray.remote(num_gpus=1)   
class Img2ImgInterface:
    def __init__(self, model_name, scheduler, use_auth_token, torch_dtype = torch.float32, safe_mode = False, device = "cuda", revision = "fp16",):
        self.pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            model_name, 
            scheduler = scheduler,  
            torch_dtype=torch_dtype,  
            use_auth_token = use_auth_token,
            # revision=revision, 
        )
        self.pipe.safety_checker = None

        self.pipe.enable_model_cpu_offload()
        self.pipe.enable_attention_slicing(1)
        self.pipe.unet.to(memory_format=torch.channels_last)
        # self.pipe.enable_vae_slicing() #Not working with img2img stable diffusion pipeline
        # self.pipe.enable_vae_tiling() #Not working with img2img stable diffusion pipeline
        self.pipe.enable_xformers_memory_efficient_attention()

    def remove_hooks(self):
        self.pipe.disable_xformers_memory_efficient_attention()
        self.pipe.disable_attention_slicing()
        remove_hook_from_submodules(self.pipe.vae)
        remove_hook_from_submodules(self.pipe.text_encoder)
        remove_hook_from_submodules(self.pipe.unet)

    def name(self):
        return "Img2ImgInterface"
    
    def __call__(
            self, 
            prompt = "",  
            negative_prompt = "", 
            image = None, 
            height = 64, 
            width = 64,  
            num_images_per_prompt = 1, 
            num_inference_steps = 50, 
            eta=0.1,
            strength=0.85,
            guidance_scale = 7.5, 
            seed = None):
        
        g_cuda = None
        if seed is not None:
            g_cuda = torch.Generator(device='cuda')
            g_cuda.manual_seed(seed)

        return self.pipe(
            prompt,
            negative_prompt = negative_prompt,
            image = image.resize((width, height)),
            num_images_per_prompt=num_images_per_prompt,
            num_inference_steps=num_inference_steps,
            eta=eta,
            strength=strength,
            guidance_scale=guidance_scale,
            generator = g_cuda
        ).images

In [10]:
@ray.remote(num_gpus=1)   
class InpaintInterface:
    def __init__(self, model_name = "runwayml/stable-diffusion-inpainting", scheduler = None, use_auth_token = None, torch_dtype = torch.float32, safe_mode = False, device = "cuda", revision = "fp16"):
        self.pipe = StableDiffusionInpaintPipeline.from_pretrained(
            model_name, 
            scheduler = scheduler,  
            torch_dtype=torch_dtype,  
            use_auth_token = use_auth_token,
        )
        self.pipe.safety_checker = None

        self.pipe.enable_model_cpu_offload()
        self.pipe.enable_attention_slicing(1)
        self.pipe.unet.to(memory_format=torch.channels_last)
        # self.pipe.enable_vae_slicing() #Not working with inpaint stable diffusion pipeline
        # self.pipe.enable_vae_tiling() #Not working with inpaint stable diffusion pipeline
        self.pipe.enable_xformers_memory_efficient_attention()

    
    def remove_hooks(self):
        self.pipe.disable_xformers_memory_efficient_attention()
        self.pipe.disable_attention_slicing()
        remove_hook_from_submodules(self.pipe.vae)
        remove_hook_from_submodules(self.pipe.text_encoder)
        remove_hook_from_submodules(self.pipe.unet)

    def name(self):
        return "InpaintInterface"
    
    def __call__(
            self, 
            prompt = "",  
            negative_prompt = "", 
            image = None, 
            mask_image = None,
            height = 64, 
            width = 64,  
            num_images_per_prompt = 1, 
            num_inference_steps = 50, 
            eta=0.1,
            guidance_scale = 7.5, 
            seed = None):
        
        g_cuda = None
        if seed is not None:
            g_cuda = torch.Generator(device='cuda')
            g_cuda.manual_seed(seed)

        return self.pipe(
            prompt,
            negative_prompt = negative_prompt,
            image = image.resize((width, height)),
            mask_image = mask_image.resize((width, height)),
            height = height,
            width = width,
            num_images_per_prompt=num_images_per_prompt,
            num_inference_steps=num_inference_steps,
            eta=eta,
            guidance_scale=guidance_scale,
            generator = g_cuda
        ).images

In [11]:
@ray.remote(num_gpus=1)   
class Depth2ImgInterface:
    def __init__(self, model_name = "stabilityai/stable-diffusion-2-depth", scheduler = None, use_auth_token = None, torch_dtype = torch.float32, safe_mode = False, device = "cuda", revision = "fp16"):
        self.pipe = StableDiffusionDepth2ImgPipeline.from_pretrained(
            model_name, 
            scheduler = scheduler,  
            torch_dtype=torch_dtype,  
            use_auth_token = use_auth_token,
        )
        self.pipe.safety_checker = None

        # self.pipe.enable_model_cpu_offload()
        self.pipe.enable_sequential_cpu_offload()
        self.pipe.enable_attention_slicing(1)
        self.pipe.unet.to(memory_format=torch.channels_last)
        # self.pipe.enable_vae_slicing() #Not working with upscale stable diffusion pipeline
        # self.pipe.enable_vae_tiling() #Not working with upscale stable diffusion pipeline
        self.pipe.enable_xformers_memory_efficient_attention()

    
    def remove_hooks(self):
        self.pipe.disable_xformers_memory_efficient_attention()
        self.pipe.disable_attention_slicing()
        remove_hook_from_submodules(self.pipe.vae)
        remove_hook_from_submodules(self.pipe.text_encoder)
        remove_hook_from_submodules(self.pipe.unet)

    def name(self):
        return "Depth2ImgInterface"
    
    def __call__(
            self, 
            prompt = "",  
            negative_prompt = "", 
            image = None, 
            height = 64, 
            width = 64,  
            num_images_per_prompt = 1, 
            num_inference_steps = 50, 
            eta=0.1,
            strength=0.85,
            guidance_scale = 7.5, 
            seed = None):
        
        g_cuda = None
        if seed is not None:
            g_cuda = torch.Generator(device='cuda')
            g_cuda.manual_seed(seed)

        return self.pipe(
            prompt,
            negative_prompt = negative_prompt,
            image = image.resize((width, height)),
            num_images_per_prompt=num_images_per_prompt,
            num_inference_steps=num_inference_steps,
            eta=eta,
            strength=strength,
            guidance_scale=guidance_scale,
            generator = g_cuda
        ).images

In [12]:
model_id = "SG161222/Realistic_Vision_V1.4" #"Linaqruf/anything-v3.0" #"admruul/anything-v3.0" #"stabilityai/stable-diffusion-2-1"  #"CompVis/stable-diffusion-v1-4" #"hakurei/waifu-diffusion" #"runwayml/stable-diffusion-v1-5"
vae_model_id = model_id

In [13]:
# 4. The scheduler for the diffusion process.
schedulers = {
    "EulerAncestralDiscreteScheduler": EulerAncestralDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "EulerDiscreteScheduler": EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "DDIMScheduler": DDIMScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "DPMSolverMultistepScheduler": DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "LMSDiscreteScheduler": LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "PNDMScheduler": PNDMScheduler.from_pretrained(model_id, subfolder="scheduler"),
}


In [14]:
stable_diffusion_interface = StableDiffusionInterface.remote(
    model_id,
    scheduler = schedulers["EulerAncestralDiscreteScheduler"],  
    use_auth_token = HF_TOKEN,
)

# stable_diffusion_interface = Img2ImgInterface(
#         model_id, 
#         vae, 
#         unet, 
#         text_encoder, 
#         tokenizer,
#         safety_checker = None,
#         feature_extractor = None,
#         scheduler = schedulers['DDIMScheduler'],  
#         use_auth_token = HF_TOKEN,
#     )

# cycle_diffusion_interface = CycleDiffusionInterface(
#     model_id, 
#     vae, 
#     unet, 
#     text_encoder, 
#     tokenizer,
#     safety_checker = None,
#     feature_extractor = None,
#     scheduler = schedulers["PNDMScheduler"],  
#     use_auth_token = HF_TOKEN,
# )
# cycle_diffusion_interface = None



# stable_diffusion_interface = CycleDiffusionInterface(
#             model_id, 
#             vae, 
#             unet, 
#             text_encoder, 
#             tokenizer,
#             safety_checker = None,
#             feature_extractor = None,
#             scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler"),  
#             use_auth_token = HF_TOKEN,
#         )


# images = stable_diffusion_interface(
#         prompt = "",
#         source_prompt = "",
#         image = Image.open("C:\\Users\\Rustam\\Downloads\\2023-03-26 04-02-23.913974.jpg"),
#         height = 384,
#         width = 384,
#         num_images_per_prompt = 1,
#         num_inference_steps = 30,
#         eta = 0.1,
#         strength = 0.8,
#         guidance_scale =7.5,
#         source_guidance_scale = 1,
#         seed = None,
#     )

# print("Done")

In [17]:
import gradio as gr



# save_images = True
save_path = "D:\Code\Diffusion_Images"

def save_images(images, save_path):
              
    try:
        for image in images:
            # display(image)
            if len(save_path) > 0:
                filename = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f').replace(":", "-") #pytz.timezone('Europe/Moscow')
                image.save(f"{save_path}/{filename}.jpg")
    except:
        print("Couldn't save image")





def generate_images(
        prompt, 
        negative_prompt = "", 
        num_samples = 1, 
        guidance_scale = 7.5, 
        num_inference_steps = 25, 
        height = 512, 
        width = 512, 
        seed = None, 
        save_path = "",
        scheduler_name = "EulerAncestralDiscreteScheduler",
    ):

    global stable_diffusion_interface
    
    if ray.get(stable_diffusion_interface.name.remote())  != "StableDiffusionInterface":
        # print(stable_diffusion_interface.__class__.__name__)
        # stable_diffusion_interface.remove_hooks()
        # del stable_diffusion_interface
        # torch.cuda.empty_cache()
        # gc.collect()
        # device = numba_cuda.get_current_device()
        # device.reset()
        # gc.collect()
        ray.kill(stable_diffusion_interface)
       
        stable_diffusion_interface = StableDiffusionInterface.remote(
            model_id,
            scheduler = schedulers[scheduler_name],  
            use_auth_token = HF_TOKEN,
        )

    seed = None if int(seed) == -1 else abs(int(seed))


    # elif stable_diffusion_interface.__class__.__name__ == "StableDiffusionInterface":
    #     if scheduler_name != stable_diffusion_interface.pipe.scheduler.__class__.__name__:
    #             stable_diffusion_interface.pipe.scheduler = schedulers[scheduler_name]

    images = ray.get(stable_diffusion_interface.__call__.remote(
        prompt,
        negative_prompt = negative_prompt,
        height = height,
        width = width,
        num_images_per_prompt = num_samples,
        num_inference_steps = num_inference_steps,
        guidance_scale =guidance_scale,
        seed = seed,
    ))

    for image in images:
        if type(image) is np.ndarray:
            image = Image.fromarray(image)

    save_images(images, save_path)

    

    return images


def cycle_generate_images(
    prompt= "", 
    source_prompt = "",
    num_samples = 1, 
    eta=0.1,
    strength=0.85,
    guidance_scale = 7.5, 
    source_guidance_scale=1,
    num_inference_steps = 25, 
    image = None,
    height = 512, 
    width = 512, 
    seed = None, 
    save_path = "",
    scheduler_name = "DDIMScheduler", 
):
    seed = None if int(seed) == -1 else abs(int(seed))

    image = image['image']
    if type(image) is not Image.Image:
        print("Cycle Diffusion Pipeline: Couldn't open image")

        return None       

    global stable_diffusion_interface 
    
    if ray.get(stable_diffusion_interface.name.remote()) != "CycleDiffusionInterface":
        # stable_diffusion_interface.remove_hooks()
        # del stable_diffusion_interface
        # torch.cuda.empty_cache()
        # gc.collect()
        # device = numba_cuda.get_current_device()
        # device.reset()
        # gc.collect()
        ray.kill(stable_diffusion_interface)
       
        stable_diffusion_interface = CycleDiffusionInterface.remote(
                model_id, 
                scheduler = schedulers[scheduler_name],  
                use_auth_token = HF_TOKEN,
            )
    
    images = ray.get(stable_diffusion_interface.__call__.remote(
        prompt = prompt,
        source_prompt = source_prompt,
        image = image,
        height = height,
        width = width,
        num_images_per_prompt = num_samples,
        num_inference_steps = num_inference_steps,
        eta = eta,
        strength = strength,
        guidance_scale =guidance_scale,
        source_guidance_scale = source_guidance_scale,
        seed = seed,
    ))

    for image in images:
        if type(image) is np.ndarray:
            image = Image.fromarray(image)

    save_images(images, save_path)

    return images

def img2img_generate_images(
    prompt = "",
    negative_prompt = "",
    num_samples = 1,
    guidance_scale = 7.5,
    eta = 0.1,
    strength = 0.85,
    num_inference_steps = 25,
    image = None,
    height = 512,
    width = 512,
    seed = None,
    save_path = "",
    scheduler_name = "DDIMScheduler",
):
    seed = None if int(seed) == -1 else abs(int(seed))

    image = image['image']
    if type(image) is not Image.Image:
        print("Img2Img Pipeline: Couldn't open image")
        return None
    
    # if 'stable_diffusion_interface' not in globals() and 'stable_diffusion_interface' not in locals():
    global stable_diffusion_interface
    
    if ray.get(stable_diffusion_interface.name.remote()) != "Img2ImgInterface":
        # stable_diffusion_interface.remove_hooks()
        # del stable_diffusion_interface
        # torch.cuda.empty_cache()
        # gc.collect()
        # device = numba_cuda.get_current_device()
        # device.reset()
        # gc.collect()
        ray.kill(stable_diffusion_interface)

        stable_diffusion_interface = Img2ImgInterface.remote(
                model_id, 
                scheduler = schedulers[scheduler_name],  
                use_auth_token = HF_TOKEN,
            )
    
    images = ray.get(stable_diffusion_interface.__call__.remote(
        prompt = prompt,
        negative_prompt = negative_prompt,
        image = image,
        height = height,
        width = width,
        num_images_per_prompt = num_samples,
        num_inference_steps = num_inference_steps,
        eta = eta,
        strength = strength,
        guidance_scale = guidance_scale,
        seed = seed,

    ))

    for image in images:
        if type(image) is np.ndarray:
            image = Image.fromarray(image)

    save_images(images, save_path)

    return images

def inpaint_generate_images(
    prompt = "",
    negative_prompt = "",
    num_samples = 1,
    guidance_scale = 7.5,
    eta = 0.1,
    num_inference_steps = 25,
    image = None,
    height = 512,
    width = 512,
    seed = None,
    save_path = "",
    scheduler_name = "DDIMScheduler",
):
    seed = None if int(seed) == -1 else abs(int(seed))

    image, mask_image = image['image'], image['mask']
    # print(image.size, mask_image.size)
    if type(image) is not Image.Image:
        print("Inpaint Pipeline: Couldn't open image")
        return None

    global stable_diffusion_interface

    
    
    if ray.get(stable_diffusion_interface.name.remote()) != "InpaintInterface":
        # stable_diffusion_interface.remove_hooks()
        # del stable_diffusion_interface
        # torch.cuda.empty_cache()
        # gc.collect()
        # device = numba_cuda.get_current_device()
        # device.reset()
        # gc.collect()
        ray.kill(stable_diffusion_interface)

        stable_diffusion_interface = InpaintInterface.remote(
                "runwayml/stable-diffusion-inpainting",
                scheduler = schedulers[scheduler_name],  
                use_auth_token = HF_TOKEN,
            )
    
    images = ray.get(stable_diffusion_interface.__call__.remote(
        prompt = prompt,
        negative_prompt = negative_prompt,
        image = image,
        mask_image = mask_image,
        height = height,
        width = width,
        num_images_per_prompt = num_samples,
        num_inference_steps = num_inference_steps,
        eta = eta,
        guidance_scale = guidance_scale,
        seed = seed,

    ))

    for image in images:
        if type(image) is np.ndarray:
            image = Image.fromarray(image)

    save_images(images, save_path)

    return images

def depth2img_generate_images(
    prompt = "",
    negative_prompt = "",
    num_samples = 1,
    guidance_scale = 7.5,
    eta = 0.1,
    strength = 0.85,
    num_inference_steps = 25,
    image = None,
    height = 512,
    width = 512,
    seed = None,
    save_path = "",
    scheduler_name = "DDIMScheduler"
):
    seed = None if int(seed) == -1 else abs(int(seed))

    image, mask_image = image['image'], image['mask']
    # print(image.size, mask_image.size)
    if type(image) is not Image.Image:
        print("depth2img Pipeline: Couldn't open image")
        return None
    
    global stable_diffusion_interface

    if ray.get(stable_diffusion_interface.name.remote()) != "depth2imgInterface":
        ray.kill(stable_diffusion_interface)

        stable_diffusion_interface = Depth2ImgInterface.remote(
                "stabilityai/stable-diffusion-2-depth",
                scheduler = schedulers[scheduler_name],  
                use_auth_token = HF_TOKEN,
            ) 

    images = ray.get(stable_diffusion_interface.__call__.remote(
        prompt = prompt,
        negative_prompt = negative_prompt,
        image = image,
        height = height,
        width = width,
        num_images_per_prompt = num_samples,
        num_inference_steps = num_inference_steps,
        guidance_scale = guidance_scale,
        seed = seed,

    ))

    for image in images:
        if type(image) is np.ndarray:
            image = Image.fromarray(image)

    save_images(images, save_path)

    return images
    

def select_interface(interface_name):
    if interface_name == "Stable Diffusion pipeline":
        return {
            negative_prompt: gr.update(visible=True),
            source_prompt: gr.update(visible=False),
            scheduler: gr.update(choices=[
                "EulerAncestralDiscreteScheduler",
                "EulerDiscreteScheduler",
                "DDIMScheduler",
                "DPMSolverMultistepScheduler",
                "LMSDiscreteScheduler",
                "PNDMScheduler"
                ],
                value="EulerAncestralDiscreteScheduler", 
            ),
            eta: gr.update(visible=False),
            strength: gr.update(visible=False),
            source_guidance_scale: gr.update(visible=False),
            image_input: gr.update(visible=False),
            generate_button: gr.update(visible=True),
            cycle_generate_button: gr.update(visible=False),
            img2img_generate_button: gr.update(visible=False),
            inpaint_generate_button: gr.update(visible=False),
            depth2img_generate_button: gr.update(visible=False),
        }

    elif interface_name == "Cycle Diffusion pipeline":
        return {
            negative_prompt: gr.update(visible=False),
            source_prompt: gr.update(visible=True),
            scheduler: gr.update(choices=[
                "DDIMScheduler",
                ],
                value="DDIMScheduler",
            ),
            eta: gr.update(visible=True),
            strength: gr.update(visible=True),
            source_guidance_scale: gr.update(visible=True),
            image_input: gr.update(visible=True),
            generate_button: gr.update(visible=False),
            cycle_generate_button: gr.update(visible=True),
            img2img_generate_button: gr.update(visible=False),
            inpaint_generate_button: gr.update(visible=False),
            depth2img_generate_button: gr.update(visible=False),
        }
    elif interface_name == "Img2Img Pipeline":
        return {
            negative_prompt: gr.update(visible=True),
            source_prompt: gr.update(visible=False),
            scheduler: gr.update(choices=[
                "EulerAncestralDiscreteScheduler",
                "EulerDiscreteScheduler",
                "DDIMScheduler",
                "DPMSolverMultistepScheduler",
                "LMSDiscreteScheduler",
                "PNDMScheduler"
                ],
                value="DDIMScheduler", 
            ),
            eta: gr.update(visible=True),
            strength: gr.update(visible=True),
            source_guidance_scale: gr.update(visible=False),
            image_input: gr.update(visible=True),
            generate_button: gr.update(visible=False),
            cycle_generate_button: gr.update(visible=False),
            img2img_generate_button: gr.update(visible=True),
            inpaint_generate_button: gr.update(visible=False),
            depth2img_generate_button: gr.update(visible=False),
        }
    elif interface_name == "Inpaint Pipeline":
        return {
            negative_prompt: gr.update(visible=True),
            source_prompt: gr.update(visible=False),
            scheduler: gr.update(choices=[
                "EulerAncestralDiscreteScheduler",
                "EulerDiscreteScheduler",
                "DDIMScheduler",
                "DPMSolverMultistepScheduler",
                "LMSDiscreteScheduler",
                "PNDMScheduler"
                ],
                value="DDIMScheduler", 
            ),
            eta: gr.update(visible=True),
            strength: gr.update(visible=False),
            source_guidance_scale: gr.update(visible=False),
            image_input: gr.update(visible=True),
            generate_button: gr.update(visible=False),
            cycle_generate_button: gr.update(visible=False),
            img2img_generate_button: gr.update(visible=False),
            inpaint_generate_button: gr.update(visible=True),
            depth2img_generate_button: gr.update(visible=False),
        } 
    elif interface_name == "Depth2Img Pipeline":
        return {
            negative_prompt: gr.update(visible=True),
            source_prompt: gr.update(visible=False),
            scheduler: gr.update(choices=[
                "EulerAncestralDiscreteScheduler",
                "EulerDiscreteScheduler",
                "DDIMScheduler",
                "DPMSolverMultistepScheduler",
                "LMSDiscreteScheduler",
                "PNDMScheduler"
                ],
                value="DDIMScheduler", 
            ),
            eta: gr.update(visible=True),
            strength: gr.update(visible=True),
            source_guidance_scale: gr.update(visible=False),
            image_input: gr.update(visible=True),
            generate_button: gr.update(visible=False),
            cycle_generate_button: gr.update(visible=False),
            img2img_generate_button: gr.update(visible=False),
            inpaint_generate_button: gr.update(visible=False),
            depth2img_generate_button: gr.update(visible=True),
        }


with gr.Blocks() as demo:

    with gr.Column():
        gr.Markdown("Hugging Face Stable Diffusion")
        save_path = gr.Textbox(label="Save path", value = save_path)

    interfaces_box = gr.Radio(
        label="Interface", 
        choices = [
            "Stable Diffusion pipeline", 
            "Cycle Diffusion pipeline",
            "Img2Img Pipeline",
            "Inpaint Pipeline",
            "Depth2Img Pipeline",
        ], 
        value = "Stable Diffusion pipeline")

    prompt = gr.Textbox(label="Prompt")

    negative_prompt = gr.Textbox(label="Negative Prompt", visible=True)
    source_prompt = gr.Textbox(label="Source Prompt", visible=False)


    num_samples = gr.inputs.Slider(label="Number of samples", default = 1, step = 1, minimum = 1, maximum = 4)
    height = gr.inputs.Slider(label="Height", default = 512, step = 64, minimum = 64, maximum = 1024)
    width = gr.inputs.Slider(label="Width", default = 512, step = 64, minimum = 64, maximum = 1024)

    eta = gr.Slider(label="Eta", value = 0.1, step = 0.1, minimum = 0.1, maximum = 1.0, visible=False)
    strength = gr.Slider(label="Strength", value = 0.85, step = 0.05, minimum = 0.05, maximum = 1.0, visible=False)
    
    with gr.Row():
        
        with gr.Column():
            scheduler = gr.inputs.Dropdown(
                label="Scheduler", 
                choices=[
                    "EulerAncestralDiscreteScheduler", 
                    "EulerDiscreteScheduler", 
                    "DDIMScheduler",
                    "DPMSolverMultistepScheduler", 
                    "LMSDiscreteScheduler", 
                    "PNDMScheduler"
                ],
                default = "EulerAncestralDiscreteScheduler"
            )
            num_inference_steps = gr.inputs.Slider(label="Number of inference steps", default = 25, step = 1, minimum = 1, maximum = 100)
            guidance_scale = gr.inputs.Slider(label="Guidance scale", default = 7.5, step = 0.1, minimum = 0.1, maximum = 10.0)
            source_guidance_scale = gr.Slider(label="Source guidance scale", value = 1, step = 0.1, minimum = 0.1, maximum = 10.0, visible=False)
            
            seed = gr.inputs.Number(label="Seed", default = -1)
            generate_button = gr.Button("Generate")
            cycle_generate_button = gr.Button("Generate", visible=False)
            img2img_generate_button = gr.Button("Generate", visible=False)
            inpaint_generate_button = gr.Button("Generate", visible=False)
            depth2img_generate_button = gr.Button("Generate", visible=False)

        image_input = gr.Image(type="pil", tool='sketch', visible=False)
        image_output = gr.Gallery(show_label=False).style(grid=[2], height="auto", preview = True)

            
    interfaces_box.change(
        select_interface, 
        interfaces_box, 
        [
            negative_prompt, 
            source_prompt, 
            scheduler,
            eta,
            strength,
            source_guidance_scale,
            image_input,
            generate_button,
            cycle_generate_button,
            img2img_generate_button,
            inpaint_generate_button,
            depth2img_generate_button
        ]
    )

    generate_button.click(
        generate_images, 
        inputs = [
            prompt, 
            negative_prompt,
            num_samples,
            guidance_scale,
            num_inference_steps,
            height,
            width,
            seed,
            save_path,
            scheduler
        ], 
        outputs=image_output
    )

    cycle_generate_button.click(
        cycle_generate_images,
        inputs = [
            prompt,
            source_prompt,
            num_samples,
            eta,
            strength,
            guidance_scale,
            source_guidance_scale,
            num_inference_steps,
            image_input,
            height,
            width,
            seed,
            save_path,
            scheduler
        
        ],
        outputs=image_output
    )

    img2img_generate_button.click(
        img2img_generate_images,
        inputs = [
            prompt,
            negative_prompt,
            num_samples,
            guidance_scale,
            eta,
            strength,
            num_inference_steps,
            image_input,
            height,
            width,
            seed,
            save_path,
            scheduler
        ],
        outputs=image_output
    )

    inpaint_generate_button.click(
        inpaint_generate_images,
        inputs = [
            prompt,
            negative_prompt,
            num_samples,
            guidance_scale,
            eta,
            num_inference_steps,
            image_input,
            height,
            width,
            seed,
            save_path,
            scheduler
        ],
        outputs=image_output
    )

    depth2img_generate_button.click(
        depth2img_generate_images,
        inputs = [
            prompt,
            negative_prompt,
            num_samples,
            guidance_scale,
            eta,
            strength,
            num_inference_steps,
            image_input,
            height,
            width,
            seed,
            save_path,
            scheduler
        ],
        outputs=image_output
    )
    

demo.queue()
demo.launch(share = True)

d:\Programs\Python\Python38\lib\site-packages\gradio\inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
d:\Programs\Python\Python38\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
d:\Programs\Python\Python38\lib\site-packages\gradio\inputs.py:217: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
d:\Programs\Python\Python38\lib\site-packages\gradio\inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://9b20780c43c9ae7cf7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(pid=10464) A matching Triton is not available, some optimizations will not be enabled.
(pid=10464) Error caught was: No module named 'triton'
(Depth2ImgInterface pid=10464) d:\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\Code\Huggingface_cache. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
(Depth2ImgInterface pid=10464) To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-d

In [16]:
#Free runtime memory
# exit()
print("Done")
# ray.kill(stable_diffusion_interface)
# del stable_diffusion_interface
# torch.cuda.empty_cache()
# gc.collect()
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()
# gc.collect()
# demo.close()

Done


(StableDiffusionInterface pid=8664) `text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
